# Downloading a dataset

In [ ]:
!pip install kaggle
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset

# Unziping it

In [ ]:
!unzip new-plant-diseases-dataset.zip -d plants

# training the dataset

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.applications.efficientnet import preprocess_input

# Paths to your dataset
train_dir = '/content/plants/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'
val_dir = '/content/plants/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'


# Parameters
img_size = 224  # EfficientNet requires images of size 224x224
batch_size = 32

# Data augmentation for training data
train_datagen = ImageDataGenerator(

    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Simple rescaling for validation data
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Load data
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)


val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)


# Load pre-trained EfficientNetB0 model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

# Initially freeze the base model
base_model.trainable = False
num_classes = len(train_data.class_indices)
# Add new layers on top of the base model
model = tf.keras.Sequential([
    base_model,  # Pre-trained EfficientNetB0 base
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

# Compile and train with frozen base model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit( train_data, epochs=5, validation_data=val_data)

# Now unfreeze the base model for fine-tuning
base_model.trainable = True

# Set a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Continue training with a lower learning rate
model.fit(train_data, epochs=5, validation_data=val_data)

model.summary()


# save and load the model

In [ ]:
model.save('plant_disease_model.keras')
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the saved model
model = tf.keras.models.load_model('plant_disease_model.keras')


# test the model

In [ ]:

# Path to the new image for prediction
img_path = '/content/plants/test/test/CornCommonRust1.JPG'  # Replace with your image path

# Resize and preprocess the image
img_size = (224, 224)  # Match model input size
img = image.load_img(img_path, target_size=img_size)  # Load and resize the image
img_array = image.img_to_array(img)  # Convert the image to an array
img_array = preprocess_input(img_array)  # Normalize the pixel values (same as during training)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Make a prediction with the model
predictions = model.predict(img_array)

# Get the predicted class index
predicted_class = np.argmax(predictions, axis=1)[0]

# Map predicted class index to the corresponding class name
class_labels = list(train_data.class_indices.keys())  # List of class names
predicted_label = class_labels[predicted_class]

# Print the predicted label
print(f"Predicted Class: {predicted_label}")
